In [110]:
import csv

# Đường dẫn đến tệp văn bản đầu vào
input_txt_file = '/home/tuandinh/Desktop/Signal Classification/data_signal_classification.txt'

# Đường dẫn đến tệp CSV đầu ra
output_csv_file = '/home/tuandinh/Desktop/Signal Classification/data.csv'

# Mở tệp văn bản đầu vào để đọc
with open(input_txt_file, 'r') as txtfile:
    # Đọc từng dòng trong tệp văn bản
    lines = txtfile.readlines()

# Mở tệp CSV đầu ra để viết
with open(output_csv_file, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(['Frequency', 'No_defect', 'with_defect'])

    # Viết dữ liệu từ tệp văn bản vào tệp CSV
    for line in lines:
        # Tách dữ liệu bằng dấu phân tách của bạn (ví dụ: dấu tab '\t')
        data = line.strip().split('\t')

        # Viết dữ liệu vào tệp CSV
        csvwriter.writerow(data)

print(f"Chuyển đổi thành công từ {input_txt_file} thành {output_csv_file}")


Chuyển đổi thành công từ /home/tuandinh/Desktop/Signal Classification/data_signal_classification.txt thành /home/tuandinh/Desktop/Signal Classification/data.csv


In [111]:
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import pandas as pd

In [112]:
class SignalDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data_frame = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        """
        Return the total number of samples in the dataset.
        """
        return len(self.data_frame)

    def __getitem__(self, index):
        sample = self.data_frame.iloc[index, :].values.astype(float)

        # The last column can be the target label (0 for no defect, 1 for defect)
        # You can adjust this based on your actual data structure
        target = int(sample[-1])

        # Remove the target label from the sample
        sample = sample[:-1]

        # Convert to PyTorch tensors
        sample = torch.FloatTensor(sample)
        # print(f"Length of sample {len(sample)}")
        target = torch.tensor(target, dtype=torch.long)

        # Apply the optional data transformation
        if self.transform:
            sample = self.transform(sample)

        return sample, target

# Path to your merged CSV file
# csv_merged = '/home/tuandinh/Desktop/Signal Classification/data_signal.csv'
csv_merged = '/home/tuandinh/Desktop/Signal Classification/file_data_new.csv'
# Create an instance of the custom dataset
custom_dataset = SignalDataset(csv_merged)
# test_dataset = SignalDataset(csv_test_path)

# # Create a data loader for the dataset
# batch_size = 32
# train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size = batch_size, shuffle=True)


In [113]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Define a perceptron 
class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(SimpleClassifier, self).__init__()
        self.fc1 = nn.Linear(163, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 2)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)   
        x = self.sigmoid(x)
        return x
    

In [114]:

# Hyperparameters
input_size = 163  # Number of features in your data
hidden_size = 64  # Number of hidden units in the neural network
num_classes = 2  # Number of classes (0 for no defect, 1 for defect)
learning_rate = 0.001
num_epochs = 10

# Split your dataset into training and validation sets
train_dataset, val_dataset = train_test_split(custom_dataset, train_size=0.8,  test_size=0.2, random_state=42, shuffle=True)

# Create data loaders for training and validation
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = SimpleClassifier(input_size, hidden_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
model

SimpleClassifier(
  (fc1): Linear(in_features=163, out_features=32, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=32, out_features=2, bias=True)
  (sigmoid): Sigmoid()
)

In [115]:
def train_one_epoch(epoch_index, tb_writer):
    running_loss = 0.0
    lass_loss = 0

    for i, data in enumerate(train_loader):
        input, labels = data
        optimizer.zero_grad()
        outputs = model(input)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        if i % 1000 == 999:
            last_loss = running_loss / 1000 # loss per batch
            print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(train_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.
    return lass_loss

In [116]:
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
writer = SummaryWriter('runs/singnal_classification_trainer_{}'.format(timestamp))
epoch_number = 0
best_vloss = 1_000_000.

for n in range(num_epochs):
    print('EPOCH {}:'.format(epoch_number + 1))
    # Make sure gradient tracking is on, and do a pass over the data
    model.train(True)
    avg_loss = train_one_epoch(epoch_number, writer)
    # validation loss
    running_vloss = 0.0
    model.eval()
    # Disable gradient computation and reduce memory consumption.
    with torch.no_grad():
        for i, vdata in enumerate(val_loader):
            val_input, vlabels = vdata
            voutputs = model(val_input)
            vloss = criterion(voutputs, vlabels)
            running_vloss += vloss
        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))
    
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss },
                    epoch_number + 1)
    writer.flush()
    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = 'models/model_{}_{}'.format(timestamp, epoch_number)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

EPOCH 1:
LOSS train 0 valid 0.6931464076042175
EPOCH 2:
LOSS train 0 valid 0.6931464076042175
EPOCH 3:
LOSS train 0 valid 0.6931464076042175
EPOCH 4:
LOSS train 0 valid 0.6931464076042175
EPOCH 5:
LOSS train 0 valid 0.6931464076042175
EPOCH 6:
LOSS train 0 valid 0.6931464076042175
EPOCH 7:
LOSS train 0 valid 0.6931464076042175
EPOCH 8:
LOSS train 0 valid 0.6931464076042175
EPOCH 9:
LOSS train 0 valid 0.6931464076042175
EPOCH 10:
LOSS train 0 valid 0.6931464076042175


In [117]:
data = pd.read_csv("/home/tuandinh/Desktop/Signal Classification/test_only_defect.csv")
# data.shape
sample = torch.tensor(data.iloc[1, :])

PATH = "/home/tuandinh/Desktop/Signal Classification/models/model_20230910_085122_0"
model1 = SimpleClassifier(163, 32,2)
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint)
model.eval()
# Thực hiện dự đoán
with torch.no_grad():
    predicted = model(sample)

/tmp/ipykernel_8197/3092540886.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  sample = torch.tensor(data.iloc[1, :])


RuntimeError: expected scalar type Double but found Float